# Filtro de Spam utilizando machine learning
Neste trabalho iremos comparar dois métodos de machine learning para classificação de emails spam.
# Coletando os dados
Primeiro iremos coletar os dados pelo site https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection arquivo 'SMSSpamCollection' e organiza-los em forma de tabela(matriz).

In [ ]:
import pandas as pd
df = pd.read_table("SMSSpamCollection",sep='\t', names =  ['label', 'sms_message'])
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Para iplementar os métodos iremos modificar os dados. Primeiro vamos tornar a variável label em um inteiro.

In [ ]:
new_values = {'spam': 1, 'ham': 0}
df.label = df.label.map(new_values)
df.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Bag of Words
Iremos modificar os dados utilizando o 'Bag of Words'(BoW), e esses novos dados serão passados aos métodos.

Bag of Words consiste em contar a frequência de uma palavra em uma frase (no contexto de programação as palavras são strings, e as frases são elementos de uma lista). Para essa contagem precisamos "padronizar" as palavras, primeiro deixaremos todas com letras minúsculas:  

In [ ]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

#lower_case_documents = list(pd.Series(documents).str.lower()) (outra forma de deixar as letras minúsculas)
#print(lower_case_documents)
lower_case_documents= []
for i in documents:
     lower_case_documents.append(i.lower())
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


Também iremos tirar pontuações, caracteres especiais e separar em forma de lista as palavras de cada frase, obtendo uma nova lista em que cada elemento da nova lista é apenas uma palavra, utilizando o código:

In [ ]:

import string

sans_punctuation_documents = [''.join(c for c in s if c not in string.punctuation) for s in lower_case_documents]
print(sans_punctuation_documents)

preprocessed_documents = []
for i in sans_punctuation_documents:
   preprocessed_documents.append(i.split(' '))
print(preprocessed_documents)

['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']
[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


Com isso podemos contar a frequência das palavras em cada frase, ou seja, temos a função Bag of Words:

In [ ]:
frequency_list = []
import pprint
from collections import Counter
for i in preprocessed_documents:
    frequency_list.append(Counter(i))
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


Apesar de implementarmos o BoW, iremos usar scikit-learn para modificar nossos dados da seguinte forma

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

count_vector.fit(documents)
count_vector.get_feature_names_out() #aqui uma pequena mudança no comando  get_feature_names()

array(['are', 'call', 'from', 'hello', 'home', 'how', 'me', 'money',
       'now', 'tomorrow', 'win', 'you'], dtype=object)

Com isso podemos criar a matriz de frequência das palavras

In [ ]:
x = count_vector.transform(documents)
doc_array = x.toarray()
frequency_matrix = pd.DataFrame(doc_array, index=documents,columns=count_vector.get_feature_names_out())
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
"Hello, how are you!",1,0,0,1,0,1,0,0,0,0,0,1
"Win money, win from home.",0,0,1,0,1,0,0,1,0,0,2,0
Call me now.,0,1,0,0,0,0,1,0,1,0,0,0
"Hello, Call hello you tomorrow?",0,1,0,2,0,0,0,0,0,1,0,1


# Conjunto de treinamento e conjunto teste
Para treinarmos os nossos métodos iremos dividir nossos dados da seguinte forma:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'],
                                                    df['label'],
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


Com isso iremos utilizar Bag of Words para modificar nosso dados para a forma desejada (matriz de frequência das palavras).

In [ ]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

# Bayes e Naive Bayes

Antes de fazermos a implementação dos métodos, iremos primeiro implementar o teorema de Bayes no problema de testes de diabetes. Neste caso como é uma classificação binária (ter ou não diabetes), estaremos utilizando os conceitos de especificidade e sensitividade.

A sensitivadade irá medir a capacidade de classificar corretamente os casos positivos, nesse caso 90% dos testes em que a pessoa possui diabetes são positivos.

Já a especificidade irá medir a capacidade de classificar corretamente os casos negativos, nesse caso 90% dos testes em que a pessoa não possui diabetes são negativos.

Ter uma sensitividade alta é mais adequado para casos em que é mais importante obter verdadeiro positivo, por exemplo na detecção de uma doença grave, pois um falso negativo pode trazer sérias consequências para o paciente.

Já uma alta especificidade é mais adequado quando os falsos positivos podem gerar maiores riscos ou consequências mais graves do que os falsos negativos, por exemplo em testes para doenças raras ou condições cujo tratamento é muito agressivo.


Para o problema de diabetes estaremos utilizando os seguintes parâmetros:

In [ ]:
# P(D) prob de ter diabetes
p_diabetes = 0.01

# P(~D) prob de não ter diabetes
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D) prob de dar pos dado que tem diabetes
p_pos_diabetes = 0.9

# Specificity or P(Neg/~D)  prob de dar neg dado que tem não diabetes
p_neg_no_diabetes = 0.9

The probability of getting a positive test result P(Pos) is: {} 0.10799999999999998
Probability of an individual having diabetes, given that that individual got a positive test result is: 0.08333333333333336
Probability of an individual not having diabetes, given that that individual got a positive test result is: {} 0.9166666666666669


Com isso podemos calcular a seguintes probabilidades

In [ ]:
# P(Pos)
p_pos = p_diabetes*p_pos_diabetes+p_no_diabetes*(1-p_neg_no_diabetes)
print('The probability of getting a positive test result P(Pos) is: {}',format(p_pos))

p_diabetes_pos = (p_diabetes*p_pos_diabetes)/p_pos
print('Probability of an individual having diabetes, given that that individual got a positive test result is:\
',format(p_diabetes_pos))

# P(Pos/~D)
p_pos_no_diabetes = 0.1
# P(~D|Pos)
p_no_diabetes_pos = (p_no_diabetes*p_pos_no_diabetes)/p_pos
print('Probability of an individual not having diabetes, given that that individual got a positive test result is: {}',format(p_no_diabetes_pos))

Note que a probabilidade de ter um teste positivo é de 10.7%, apesar da probabilidade de alguém possuir diabetes ser de 1%, o que pode parecer um pouco contra intuitivo. Também pode ser contra intuitivo ver que a chance de uma pessoa ter diabetes dado que o teste é postivo é de apenas 8.7%.

A probabilidade de não ter diabetes dado que o teste deu positivo é de 91%, o que nos leva a pergunta: Vale a pena mesmo fazer o teste de diabetes sem nenhuma outra informação adicional??

Agora, implementaremos o Naive Bayes em palavras usadas em debates por dois participantes, para calcular a probabilidade de duas palavras serem usadas durante os discursos. Os parâmetros foram:

In [ ]:
# P(J) prob de J fazer o discurso
p_j = 0.5

# P(F/J) prob de falar F dado que fez o disc
p_j_f = 0.1

# P(I/J) prob de falar I dado que fez o disc
p_j_i = 0.1

p_j_text = p_j*p_j_f*p_j_i
print(p_j_text)

# P(G)
p_g = 0.5

# P(F/G)
p_g_f = 0.7

# P(I/G)
p_g_i = 0.2

p_g_text = p_g*p_g_f*p_g_i
print(p_g_text)

p_f_i = p_g_text+p_j_text
print('Probability of words freedom and immigration being said are: ', format(p_f_i))

0.005000000000000001
0.06999999999999999
Probability of words freedom and immigration being said are:  0.075


Também podemos calcular a probabilidade de J falar as palavras F e I, e de G falar as palvras F e I

In [ ]:
p_j_fi = (p_j*p_j_f*p_j_i)/p_f_i
print('The probability of Jill Stein saying the words Freedom and Immigration: ', format(p_j_fi))

p_g_fi = (p_g*p_g_f*p_g_i)/p_f_i
print('The probability of Gary Johnson saying the words Freedom and Immigration: ', format(p_g_fi))

The probability of Jill Stein saying the words Freedom and Immigration:  0.06666666666666668
The probability of Gary Johnson saying the words Freedom and Immigration:  0.9333333333333332


# Aplicando o método de Naive Bayes para identificar emails spam
Apesar de implementarmos o Naive Bayes, utilizaremos sklearn.naive_bayes para fazer previsões com base nos nossos dados de  treinamento da seguinte forma:  

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()

Criaremos uma variável para as previsões do Naive Bayes no nosso conjunto de dados teste

In [ ]:
predictions = naive_bayes.predict(testing_data)

Finalmente podemos comparar nossas previsões com o nosso conjunto de dados teste. Iremos fazer essa comparação utilizando as funções accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439561


Esses são os resultados utilizando o método de Naive Bayes, agora iremos comparar esses resultados com as previsões obtidas com o método K-Nearest-Neighbors(KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(training_data, y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
knnprediction = knn.predict(testing_data)
print('Accuracy score: ', format(accuracy_score(y_test, knnprediction)))
print('Precision score: ', format(precision_score(y_test, knnprediction)))
print('Recall score: ', format(recall_score(y_test, knnprediction)))
print('F1 score: ', format(f1_score(y_test, knnprediction)))

Accuracy score:  0.9030868628858578
Precision score:  1.0
Recall score:  0.2702702702702703
F1 score:  0.425531914893617


Podemos criar uma tabela pra uma melhor vizualização dos resultados e compará-los

In [ ]:
a = accuracy_score(y_test, predictions)
b = precision_score(y_test, predictions)
c = recall_score(y_test, predictions)
d = f1_score(y_test, predictions)
e = accuracy_score(y_test, knnprediction)
f = precision_score(y_test, knnprediction)
g = recall_score(y_test, knnprediction)
h = f1_score(y_test, knnprediction)
from tabulate import tabulate

tabela_de_comparacao = [
    ["Accuracy", a, e],
    ["Precision", b, f],
    ["Recall", c, g],
    ["F1score", d, h]
]
head = ["Medição", "NB", "K-NN"]
print(tabulate(tabela_de_comparacao, headers=head, tablefmt="grid"))

+-----------+----------+----------+
| Medição   |       NB |      KNN |
+===========+==========+==========+
| Accuracy  | 0.988514 | 0.903087 |
+-----------+----------+----------+
| Precision | 0.972067 | 1        |
+-----------+----------+----------+
| Recall    | 0.940541 | 0.27027  |
+-----------+----------+----------+
| F1score   | 0.956044 | 0.425532 |
+-----------+----------+----------+


Primeiro comparando a Accuracy, vemos que o NB se mostra mais preciso nas classificações, provavelmente devido ao fato que o K-NN está calculando o vizinho mais próximo utilizando distância euclidiana de vetores, em que os vetores são as linhas da matriz de frequência.

A precisão do K-NN é 1, ou seja, toda mensagem classificada como spam é realmente spam, provavelmente do fato da métrica utilizada para encontrar os vizinhos mais próximos.

O Recall foi de 0.27, então o método K-NN, apesar de ter uma precisão de 100%, classificou 73% dos emails que eram spam como não spam, ou seja, o método possui um desempenho ruim em classificar emails que são spam. Uma teoria do porque isso estar acontecendo é que provavelmente o método, está apenas conseguindo classificar emails como spam, quando é um email muito "óbvio" de ser spam (em que esse óbvio é relativo ao método utilizado).  

E finalmente comparando o F1score, é fácil de ver que o Naive Bayes tem um desempenho muito melhor que o K-NN, que vem do fato do método K-NN não conseguir classificar de forma mais precisa os emails que são spam, ou seja, ele classifica a maioria dos emails como não spam, o que pode ser um problema em casos práticos.

Agora comentando um pouco sobre os prós e contras do K-NN. Um contra é que como no nosso problema temos que cada linha (vetor) tem n colunas, em que n é o número de palavras em todos os email e o K-NN é bastante impactado pela questão da dimensão do problema (até pela questão da memória utilizada), isso pode ter gerado problemas ao K-NN e por isso ter um F1score tão menor que o NB. Mas tem um pró que a questão de ele ser simples e fácil de interpretar os resultados.

Os contras do Naive Bayes é que ele assume uma distribuição de probabilidade que pode não existir, e assume que as "features" do problema são independetes (que é o caso para o problema de frequência de palarvas em emails spam). Os prós é que ele funciona bem para problemas que possuem "features" independentes, e é fácil de implementar, como fizemos nesse notebook no problema de testes de diabetes.   


# Conclusão
Com isso encerramos esse projeto, e com ele podemos concluir que o método de Naive Bayes se mostra mais eficiente que o método K-Nearest-Neighbors no problema de classificação de mensagens spam. A razão disso, pode ser pelo fato que do NB ter uma abordagem paramétrica, e o K-NN uma abordagem não paramétrica. Para um estudo futuro poderíamos utilizar outros valores de n_neighbors no método K-NN (nesta caso usamos 7), e outras métricas para obter diferente resultados.